In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

# Create PyTorch datasets and data loaders
train_dataset = TensorDataset(X_train)
test_dataset = TensorDataset(X_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 2  # Latent space dimension
model = Autoencoder(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    train_loss = 0.0
    for data in train_loader:
        inputs = data[0]
        optimizer.zero_grad()
        _, outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}')

# Save the trained model weights
torch.save(model.state_dict(), 'trained_model.pth')

# Extract the latent space representations
latent_representations = []
labels = []
with torch.no_grad():
    for data, target in zip(test_loader, y_test):
        inputs = data[0]
        encoded, _ = model(inputs)
        latent_representations.append(encoded.numpy())
        labels.append(iris.target_names[target])
        print(target)
        #labels.extend([iris.target_names[label] for label in target])

# Concatenate the latent representations and labels
latent_representations = np.concatenate(latent_representations)
#labels = np.array(labels)


Epoch 1/100, Train Loss: 1.317678
Epoch 2/100, Train Loss: 1.316481
Epoch 3/100, Train Loss: 1.315292
Epoch 4/100, Train Loss: 1.314183
Epoch 5/100, Train Loss: 1.313030
Epoch 6/100, Train Loss: 1.311960
Epoch 7/100, Train Loss: 1.310871
Epoch 8/100, Train Loss: 1.309776
Epoch 9/100, Train Loss: 1.308748
Epoch 10/100, Train Loss: 1.307732
Epoch 11/100, Train Loss: 1.306732
Epoch 12/100, Train Loss: 1.305681
Epoch 13/100, Train Loss: 1.304705
Epoch 14/100, Train Loss: 1.303746
Epoch 15/100, Train Loss: 1.302777
Epoch 16/100, Train Loss: 1.301811
Epoch 17/100, Train Loss: 1.300886
Epoch 18/100, Train Loss: 1.299971
Epoch 19/100, Train Loss: 1.299011
Epoch 20/100, Train Loss: 1.298145
Epoch 21/100, Train Loss: 1.297255
Epoch 22/100, Train Loss: 1.296344
Epoch 23/100, Train Loss: 1.295458
Epoch 24/100, Train Loss: 1.294591
Epoch 25/100, Train Loss: 1.293718
Epoch 26/100, Train Loss: 1.292877
Epoch 27/100, Train Loss: 1.292031
Epoch 28/100, Train Loss: 1.291168
Epoch 29/100, Train Loss: 1.2

In [44]:
len(latent_representations[:, 0]),len(latent_representations[:, 1])

(30, 30)

In [50]:
latent_representations

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.08061126],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.0764538 ],
       [0.        , 0.00746146],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.01901871],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.07510528],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.

In [46]:

# Visualize the latent space
plt.figure(figsize=(8, 6))
sns.scatterplot(x=latent_representations[:, 0], y=latent_representations[:, 1], hue=labels, palette='viridis')
plt.colorbar()
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('Latent Space Representation')
plt.show()

ValueError: All arrays must be of the same length

<Figure size 800x600 with 0 Axes>

In [28]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

device(type='cuda')

In [3]:
#dataset = load_iris()


{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

(torch.Size([124, 13]), torch.Size([54, 13]))

In [9]:
D_in = len(dataset.feature_names)
H = 50
H2 = 12
model = Autoencoder(D_in, H, H2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
loss_mse = customLoss()

In [11]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []

In [12]:
model

Autoencoder(
  (linear1): Linear(in_features=13, out_features=50, bias=True)
  (lin_bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=50, out_features=12, bias=True)
  (lin_bn2): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=12, out_features=12, bias=True)
  (lin_bn3): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=12, out_features=3, bias=True)
  (bn1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc21): Linear(in_features=3, out_features=3, bias=True)
  (fc22): Linear(in_features=3, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
  (fc_bn3): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=3, out_features=12, bias=True)
  (fc_bn4): BatchNorm1d(12, eps=1e-05, mome

In [13]:

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device).to(torch.float32)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 200 == 0:        
        print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

In [14]:
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device).to(torch.float32)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 200 == 0:        
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [15]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

====> Epoch: 200 Average training loss: 13.3368
====> Epoch: 200 Average test loss: 14.1931
====> Epoch: 400 Average training loss: 11.5527
====> Epoch: 400 Average test loss: 11.8054
====> Epoch: 600 Average training loss: 9.7752
====> Epoch: 600 Average test loss: 11.9003
====> Epoch: 800 Average training loss: 8.9588
====> Epoch: 800 Average test loss: 9.8614
====> Epoch: 1000 Average training loss: 8.3919
====> Epoch: 1000 Average test loss: 9.9363
====> Epoch: 1200 Average training loss: 7.5658
====> Epoch: 1200 Average test loss: 9.6052
====> Epoch: 1400 Average training loss: 7.0090
====> Epoch: 1400 Average test loss: 8.8501


In [16]:

with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device).to(torch.float32)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        
scaler = trainloader.dataset.standardizer
recon_batch

tensor([[ 6.0125e-01, -4.1092e-01, -8.6919e-01, -1.2948e+00,  8.8519e-01,
          5.5696e-01,  7.8093e-01, -6.6191e-01,  1.0412e+00, -8.0629e-02,
          3.2283e-01,  8.9173e-01,  7.0137e-01],
        [ 1.7891e-01, -2.7480e-01,  1.9202e-01, -1.3058e-01, -2.8748e-02,
          5.5199e-01,  6.8846e-01, -6.0493e-01,  6.8843e-02, -3.1970e-01,
          3.2762e-01,  1.0095e+00,  2.3109e-01],
        [ 1.2009e-01,  9.1562e-01,  2.7403e-01,  5.2794e-01, -8.2547e-02,
         -9.3469e-01, -1.1771e+00,  6.0149e-01, -7.3297e-01,  9.4879e-01,
         -1.1487e+00, -1.2615e+00, -4.1227e-01],
        [ 7.6353e-01, -5.7202e-01,  8.7160e-01, -1.7013e-01,  1.0368e+00,
          6.1942e-01,  8.1148e-01, -2.1581e-01,  1.7758e-01,  4.4685e-02,
          6.0197e-01,  6.6050e-01,  1.1582e+00],
        [-8.8740e-01, -6.6023e-01, -8.5951e-01, -1.1303e-01, -9.3083e-01,
          5.2852e-02, -3.4167e-02, -4.3076e-01, -2.7521e-01, -8.1222e-01,
          4.3500e-01,  3.5468e-01, -8.6921e-01],
        [ 2.420

In [19]:
recon_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
real_row = scaler.inverse_transform(testloader.dataset.x[0].cpu().numpy())

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.60125285 -0.4109178  -0.8691857  -1.2948145   0.88518506  0.5569587
  0.78092647 -0.6619064   1.0412297  -0.08062924  0.3228297   0.89173126
  0.70137316].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [18]:
df = pd.DataFrame(np.stack((recon_row, real_row)), columns = dataset.feature_names)
df

ValueError: Must pass 2-d input. shape=(2, 54, 13)

In [ ]:
np.stack((recon_row, real_row))